## Marker to robot base from trajectory

In [1]:
log_folder = "/home/pjlab/Franka_Tool/frankapy/logs/2024-09-22_14-12-28/log-000002-6876"
calibration_eyeinhand_results_dir = "/home/pjlab/Franka_Tool/catkin_ws_2/src/easy_handeye/results/eye_on_hand/9-21--15-38.json"

In [ ]:
import cv2
import numpy as np
import h5py
import argparse
from pathlib import Path
import roboticstoolbox as rtb
import json
import open3d as o3d

def get_to_marker_pose(image, camera_params, tag_size, detector, min_num=4):
    '''
    Get the pose: camera -> marker
    '''
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    results = detector.detect(gray, estimate_tag_pose=True, camera_params=camera_params, tag_size=tag_size)
    if len(results) < min_num:
        return None
    res = None
    for result in results:
        if result.tag_id == 22222:
            res = result
            break
    if res is None:
        return None
    pose = np.eye(4)
    pose[:3, :3] = res.pose_R
    pose[:3, 3] = res.pose_t[:, 0]
    pose = np.linalg.inv(pose)
    hope2now = np.eye(4)
    hope2now[1, 1] = -1
    hope2now[2, 2] = -1
    now2hope = np.linalg.inv(hope2now)
    pose = now2hope @ pose 
    return pose

def create_camera_model(size=0.1):
    # Create a simple camera model (Frustum shape)
    mesh_camera = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size, origin=[0, 0, 0])
    return mesh_camera

def show_pose(camera_pose, size=0.1):
    camera_pose = np.array(camera_pose)
    camera_pose[:3, :3] = camera_pose[:3, :3] / np.abs((np.linalg.det(camera_pose[:3, :3]))) ** (1/3)
    # Apply camera pose transformation
    tmp_tans = np.eye(4)
    tmp_tans[2, 2] = -1
    camera_model = create_camera_model(size)
    camera_model.transform(camera_pose)
    return camera_model

def filter_and_compute_mean_std(matrices, outlier_percent=0.1):
    """
    Filter out a percentage of outliers from the given matrices and compute the mean and standard deviation.

    Parameters:
    matrices (numpy.ndarray): A numpy array of shape (n, m, m) representing n matrices of size m x m.
    outlier_percent (float): The percentage of outliers to remove (0 < outlier_percent < 0.5), default is 10%.

    Returns:
    mean_matrix (numpy.ndarray): The mean matrix after filtering out outliers.
    std_matrix (numpy.ndarray): The standard deviation matrix after filtering out outliers.
    """
    # Step 1: Compute the norm of each matrix
    norms = np.linalg.norm(matrices, axis=(1, 2))

    # Step 2: Sort the norms
    sorted_indices = np.argsort(norms)

    # Step 3: Compute the bounds for removing outliers
    n_matrices = len(matrices)
    lower_bound = int(n_matrices * outlier_percent)
    upper_bound = int(n_matrices * (1 - outlier_percent))

    # Step 4: Filter matrices
    filtered_indices = sorted_indices[lower_bound:upper_bound]
    filtered_matrices = matrices[filtered_indices]

    # Step 5: Compute the mean and standard deviation after removing outliers
    mean_matrix = np.mean(filtered_matrices, axis=0)
    std_matrix = np.std(filtered_matrices, axis=0)

    return mean_matrix, std_matrix


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [12]:
from scipy.spatial.transform import Rotation as R
from pupil_apriltags import Detector

log_folder = Path(log_folder)
hdf5_file = log_folder / "traj.hdf5"
hdf5_file = h5py.File(hdf5_file, 'r')
joints = hdf5_file['joints'][:]
timestamps = hdf5_file["timestamp"][:]
qposes = joints[:, :7]
franka = rtb.models.Panda()
with open(calibration_eyeinhand_results_dir, 'r') as f:
    calibration_results = json.load(f)
cam_2_ee = np.eye(4)
quat = np.array([calibration_results["rotation"]["x"], calibration_results["rotation"]["y"], calibration_results["rotation"]["z"], calibration_results["rotation"]["w"]])
cam_2_ee[:3, :3] = R.from_quat(quat, scalar_first=False).as_matrix()
cam_2_ee[:3, 3] = np.array([calibration_results["translation"]["x"], calibration_results["translation"]["y"], calibration_results["translation"]["z"]])
print(f"cam_EE:\n{cam_2_ee}")
detector =  Detector(families="tagStandard52h13",
                    nthreads=1,
                    quad_decimate=1.0,
                    quad_sigma=0.0,
                    refine_edges=1,
                    decode_sharpening=0.25,
                    debug=0)
item_to_show = []
i = 0
import sys
marker_2_bases = []
print("Total num:", len(qposes))
for qpos, timestamp in zip(qposes, timestamps):
    if i % 10 != 0 or i > 50:
        i += 1
        continue
    ee_2_base = np.array(franka.fkine(qpos))
    cam_2_base = ee_2_base @ cam_2_ee @ rtb.ET.Rz(-np.pi/2).A()
    # ee_model = show_pose(cam_2_base, 0.1)
    # item_to_show.append(ee_model)
    # print(f"timestamp: {timestamp}")
    # print(f"EE_base:\n{ee_2_base}")
    # print(f"cam_base:\n{cam_2_base}")
    image_path = log_folder / "color_1/" / f"{timestamp}.png"
    image = cv2.imread(str(image_path))
    intr = hdf5_file["intrinsic_color_1"]
    fx, fy, cx, cy = intr["fx"][()], intr["fy"][()], intr["ppx"][()], intr["ppy"][()]
    camera_params = (fx, fy, cx, cy)
    cam_2_maker = get_to_marker_pose(image, camera_params, 0.031, detector)
    # print(f"cam_marker:\n{cam_2_maker}")
    if cam_2_maker is not None:
        marker_2_base = cam_2_base @ np.linalg.inv(cam_2_maker)
        marker_2_bases.append(marker_2_base)
    if i == 0:
        print(f"Marker_base\n{marker_2_base}")
    i += 1

marker_2_bases = np.array(marker_2_bases)
mean_marker_2_base, std_marker_2_base = filter_and_compute_mean_std(marker_2_bases)
np.save(log_folder / "marker_2_base.npy", mean_marker_2_base)
print(f"mean_marker_2_base\n{mean_marker_2_base}")
print(f"std_marker_2_base\n{std_marker_2_base}")

# frame_base = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6, origin=[0, 0, 0])
# item_to_show.append(frame_base)
# o3d.visualization.draw_geometries(item_to_show)



cam_EE:
[[ 4.68582381e-04 -9.98368542e-01 -5.70967069e-02  8.01850673e-02]
 [ 9.99527826e-01  2.22178170e-03 -3.06461778e-02 -2.74386378e-02]
 [ 3.07230363e-02 -5.70553871e-02  9.97898180e-01 -8.94739605e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Total num: 1000
Marker_base
[[ 6.25873131e-02  9.97268753e-01  3.92155984e-02  4.93996254e-01]
 [-9.78507076e-01  6.90506589e-02 -1.94308798e-01  1.54858690e-01]
 [-1.96485956e-01 -2.62114749e-02  9.80156226e-01  9.38567945e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
mean_marker_2_base
[[ 6.04145342e-02  9.90460435e-01 -6.14805835e-02  4.65377361e-01]
 [-9.77241112e-01  4.70311612e-02 -2.03544594e-01  1.29829799e-01]
 [-2.01198456e-01  7.25656666e-02  9.70787614e-01 -4.16775610e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
std_marker_2_base
[[0.0032019  0.01094724 0.10690301 0.02261133]
 [0.0063622  0.0230121  0.02806139 0.02333235]
 [0.02860671 0.10421622 0.